In [16]:
from skimage.feature import daisy
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from tensorflow import keras

# import the fashion_mnist dataset from keras
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# setting the parameters for the daisy feature extractor

step = 7
radius = 3
rings = 2
histograms = 6
orientations = 8

# the length each feature vector will have
vec_length = (rings * histograms + 1) * orientations

# the number of features per image
daisies = int((train_images[0].shape[0] / step) ** 2)

def create_features(images): 

    daisy_feature = np.zeros(shape=(daisies * len(images), vec_length))

    # create the 16 features for each image
    d = 0
    for i in range(len(images)):

        # we don't need the image
        descs = daisy(images[i], step=step, radius=radius, rings=rings, histograms=histograms,
                      orientations=orientations, visualize=False)

        # reducing the dimension
        daisy_array = descs.reshape(daisies, vec_length)

        for j, feature_idx in enumerate(range(d, d + daisies)):
            daisy_feature[feature_idx] = daisy_array[j]

        d += daisies
    
    return daisy_feature


daisy_train = create_features(train_images)
daisy_test = create_features(test_images)

#implementing k-means

# number of clusters
k = 60

# randomly initialize the centroids for k means
np.random.seed(0)
initial_centroids = np.zeros(shape=(k, vec_length))
random_indices = random.sample(range(len(daisy_train)), k)

for i, rand in enumerate(random_indices):
    initial_centroids[i] = daisy_train[rand]


# get the euclidean distance between each feature and each centroid
def calculate_distance(features, centroids):
    distances = (-2 * np.dot(centroids, features.T) + np.sum(features ** 2, axis=1) + np.sum(centroids ** 2, axis=1)[:,
                                                                                      np.newaxis]).T
    return distances


# assign each feature to closest centroid

def centroid_assign_features(features, distances, k):
    # the dictionary stores as the centroids as key and the assigned features as values
    visual_dictionary = {c: [] for c in range(k)}

    for i, dis in enumerate(distances):
        # find the index of the closest cluster for each feature
        min_dis_index = np.argmin(dis)
        # append the feature to the respective cluster in the dictionary
        visual_dictionary[min_dis_index].append(features[i])

    return visual_dictionary


# calculate the new centroids

def calculate_new_centroids(visual_dictionary, vec_length, k):
    updated_centroids = np.zeros(shape=(k, vec_length))

    for i, features in enumerate(visual_dictionary.values()):
        # take the sum of all features for each cluster
        stacked_array = np.vstack(features)
        # then take the average which will be the updated centroid
        new_centroid = np.ndarray.mean(stacked_array, axis=0)
        updated_centroids[i] = new_centroid

    return updated_centroids


def CreateDictionary(initial_centroids, features, vec_length, k):
    # the first iteration starts with the randomly created centroids
    new_centroids = initial_centroids

    for i in range(500):

        # calling the above created functions
        distances = calculate_distance(features, new_centroids)

        visual_dictionary = centroid_assign_features(features, distances, k)

        updated_centroids = calculate_new_centroids(visual_dictionary, vec_length, k)

        # check if the centroids have changed since the last iteration
        if np.array_equal(new_centroids, updated_centroids):
            # if there is no change the final centroids have been found
            return visual_dictionary, distances, updated_centroids, i

        new_centroids = updated_centroids

    return visual_dictionary, distances, updated_centroids, i


visual_dictionary, all_distances_train, centroids, num_iter = CreateDictionary(initial_centroids, daisy_train,
                                                                               vec_length, k)

# find the distance of each of the test features to each of the final centroids

all_distances_test = calculate_distance(daisy_test, centroids)

# get the feature which is closest to each of the centroids
closest_index = np.argmin(all_distances_train, axis=0)
# find the image of the closest feature as well as the feature cannot be displayed as an image, since it is a vector of orientations
closest_image_index = closest_index / daisies
closest_image_index = closest_image_index.astype(int)


# create the folder in which the closest features will be stored

outdir = './cluster_features'

if not os.path.exists(outdir):
    os.mkdir(outdir)

for i, j in enumerate(closest_index):
    outname = 'cluster_feature' + str(i + 1) + '.txt'

    fullname = os.path.join(outdir, outname)

    np.savetxt(fullname, daisy_train[j])

# https://stackoverflow.com/questions/47143836/pandas-dataframe-to-csv-raising-ioerror-no-such-file-or-directory

# create the folder in which the images of the closest features will be stored
outdir = './cluster_images'
if not os.path.exists(outdir):
    os.mkdir(outdir)

for i, j in enumerate(closest_image_index):
    outname = 'cluster_image' + str(i + 1) + '.jpeg'

    fullname = os.path.join(outdir, outname)

    plt.imsave(fullname, train_images[j], cmap=plt.cm.gray)


# softmax is used to calculate the probability of an observation belonging to a class
# 'softmin' is used here as we need to give a higher probability to a feature belonging to a cluster if the distance is smaller

def softmin(x):
    # Compute softmin values for each sets of scores in x
    e_x = np.exp(np.max(x) - x)
    return e_x / e_x.sum(axis=0)


# NOTE: Different values for n for the nearest neigbour have been used, however, the best results were achieved
# by using all clusters (the probability of a feature belonging to each of the clusters)

def ComputeHistogram(train_images, test_images, k, all_distances_train, all_distances_test, daisies):
    # create an empty histogram for all images
    hist_matrix_train = np.zeros(shape=(len(train_images), k))

    # a counter used to loop through each feature of an image
    d = 0
    for i in range(len(train_images)):
        # create an empty histogram for all features of an image
        hist = np.zeros(shape=(k))
        for j in range(d, d + int(daisies)):
            # the probability for each feature to belong to a cluster is calculated by calling softmin
            hist += softmin(all_distances_train[j])
        # insert the histogram for each image to the histogram array for all images
        hist_matrix_train[i] = hist / daisies
        d += daisies

    # the same steps are conducted for the test images
    hist_matrix_test = np.zeros(shape=(len(test_images), k))

    d = 0
    for i in range(len(test_images)):
        hist = np.zeros(shape=(k))
        for j in range(d, d + daisies):
            hist += softmin(all_distances_test[j])
        hist_matrix_test[i] = hist / daisies
        d += daisies

    return hist_matrix_train, hist_matrix_test


hist_matrix_train, hist_matrix_test = ComputeHistogram(train_images, test_images, k, all_distances_train,
                                                       all_distances_test, daisies)



def histogram_intersection(hist1, hist2, widthHist):
    sumVal = 0
    for i in range(widthHist):
        # get the smaller value for each bin to calculate the amount of intersection
        sumVal += min(hist1[i], hist2[i])
    return sumVal


def MatchHistogram(hist_matrix_train, hist_matrix_test, test_labels, k):
    # this saves the indices of the train images which have the highest intersection for each test image
    best_match_indices = []

    for hist_test in hist_matrix_test:
        hist_intersec_score = 0
        for j, hist_train in enumerate(hist_matrix_train):
            # calculate the intersection for each test image with each train image
            score = histogram_intersection(hist_test, hist_train, k)
            # find the most overlapping train histogram for each test histogram
            if score > hist_intersec_score:
                hist_intersec_score = score
                # save the index of the train histogram which has the highest intersection
                best_match_index = j
        best_match_indices.append(best_match_index)

    # get the predicted labels by assigning the label of the best matched train image to the test image
    pred_labels = np.zeros(shape=(len(test_labels)))

    for i in range(len(test_labels)):
        pred_labels[i] = train_labels[best_match_indices[i]]

    return pred_labels

#NOTE: Due to computational constraints only 1000 test images are matched

pred_labels = MatchHistogram(hist_matrix_train, hist_matrix_test[0:1000], test_labels[0:1000], k)

def CalculatePerformaneMetric(test_labels, pred_labels):
    # calculate the true positive, false positive, false negative and true negative rates

    TP = np.zeros(shape=10)
    FP = np.zeros(shape=10)
    FN = np.zeros(shape=10)
    TN = np.zeros(shape=10)

    for i in range(10):
        TP[i] = np.sum((test_labels == i) * (pred_labels == i))
        FP[i] = np.sum((test_labels != i) * (pred_labels == i))
        FN[i] = np.sum((test_labels == i) * (pred_labels != i))
        TN[i] = np.sum((test_labels != i) * (pred_labels != i))

    # use the above calculated measures to get accuracy, precision and recall

    accuracy_overall = np.around(((TP) / (TP + TN + FP + FN)).sum(), 4)
    precision = np.around((TP / (TP + FP)).mean(), 4)
    recall = np.around((TP / (TP + FN)).mean(), 4)
    class_wise_accuracy = np.around(TP / (TP + FN), 4)

    return accuracy_overall, precision, recall, class_wise_accuracy

# NOTE: Since we only matched the first 1000 test images, we also will take only the first 1000 test labels
accuracy_overall, precision, recall, class_wise_accuracy = CalculatePerformaneMetric(test_labels[0:1000], pred_labels)

print('The overall accuracy is:', accuracy_overall)
print('The precision is:', precision)
print('The recall is:', recall)


'''
References:

desertnaut(2016). Answer on stack overflow to question: How to implement the Softmax function in Python? 
    Retrieved from: https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python

Dey, S.(2016). Distance Matrix Vectorization Trick.
    Retrieved from: https://medium.com/@souravdey/l2-distance-matrix-vectorization-trick-26aa3247ac6c

Melli, G.(2021). Softmin Activation Function. 
    Retrieved from: https://www.gabormelli.com/RKB/Softmin_Activation_Function

''';

The overall accuracy is: 0.744
The precision is: 0.7493
The recall is: 0.7472
